In [ ]:
from brian2 import *
from QIF import Neuron_QIF
from QIF_DOP import Neuron_QIF_DOP
from simulation_runner import run_simulation
from synapse import SynapseModel, connect
from params import load_params

In [ ]:
from brian2tools import *
import matplotlib.pyplot as plt
import random
import numpy as np
from brian2.units import *
# from neuron import h, gui
from brian2 import ms, mV, pA, pF, siemens, amp, second, NeuronGroup, defaultclock, run
import json
import numpy as np

In [ ]:
def main():
    # Define parameters for NeuronModel
    params = load_params('./QIF_DOP.json')
    N = params.pop('N')
    
    # Define v_reset
    v_reset = -66 * mV
    
    # Run the simulation
    run_simulation(N, params, v_reset)

In [ ]:
# with dopamine
if __name__ == '__main__':
    main()

In [ ]:
def main():
    # Define parameters for NeuronModel
    params = load_params('./QIF.json')
    N = params.pop('N')
    
    # Define v_reset
    v_reset = -66 * mV
    
    # Run the simulation
    run_simulation(N, params, v_reset)

In [ ]:
# withouyt dopamine
if __name__ == '__main__':
    main()

In [ ]:
from brian2 import *
from AdEx import NeuronModel_AdEx
import json
import subprocess
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
from simulation_runner import NeuronModel, Simulation, load_params, run_simulation

In [ ]:
# run_simulation.py 스크립트 실행
result = subprocess.run(['python', 'simulation_runner.py'], capture_output=True, text=True)

# 스크립트 출력 결과 출력
print(result.stdout)
if result.stderr:
    print(result.stderr)

In [ ]:

# JSON 파일 읽기
with open('./GPe_1.json', 'r') as f:
    config = json.load(f)

# JSON에서 파라미터 가져오기
model_name = config['model']
model_parameters = config['params']

# 모델 인스턴스 생성 및 실행
if model_name == "AdEx":
    model = NeuronModel_AdEx(model_parameters['N']['value'], model_parameters)
    model.run(1*second)

In [1]:
import numpy as np
from simulation_runner import load_params, run_simulation
from brian2 import mV

# JSON 파일 로드
json_file = './GPe_1.json'  # 사용할 JSON 파일 경로

# 파라미터 로드
params, model_name = load_params(json_file)
N = params.pop('N')['value']  # N은 모델의 뉴런 수
v_reset = params['vr']['value'] * mV  # vr 값을 v_reset으로 사용

# 시뮬레이션 실행
sim, earliest_time_stabilized = run_simulation(N, params, v_reset, model_name)

# 결과 시각화
sim.plot_results(earliest_time_stabilized)

Error processing parameter 'g_L': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'E_L': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'Delta_T': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'vt': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'vr': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'tau_w': only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
Error processing parameter 'a': only integers, slices (`:`), ellipsis (`...`), numpy

BrianObjectException: Error encountered with object named 'neurongroup'.
Object was created here (most recent call only, full details in debug log):
  File '/Users/tacolab4/Desktop/neuroscience/code/bg_insilico_dopamine/models/AdEx.py', line 25, in build_model
    self.neurons = NeuronGroup(self.N, model=eqs, threshold='v > vt', reset='v = E_L; u += d', method='euler')

An error occurred when preparing an object. (See above for original error message and traceback.)